# 7. Flask - Modular Architecture I: The Application Factory
Our current Mission Control system works, but it's a monolithic prototype where all systems are hardwired together. In this lesson, we will perform a major upgrade: we will learn to build the core of our application using the **Application Factory** pattern, which separates the system's construction from its execution. This will allow us to easily run our system in different configurations and prepare an external "docking port" (`wsgi.py`) for future deployment to production servers.

## 7.1. BASIC Structure - Micro-solution for an Application
Every great mission starts with a simple prototype. At this stage, we have all the logic in a single file, which is ideal for quickly verifying the functionality of a basic onboard computer.

- Everything is in `one file`.
- Configuration and routes are often `hardcoded` directly in the application file, or ideally in their own files (e.g., `config.py`, `routes.py`).

- Suitable only for `simple logic`, validating an idea, `prototyping a concept`.


`Application structure`:
- my_project/
  - my_app.py
  - static/
  - templates/
  - config/

## 7.2. STANDARD Structure - Small to Medium-Sized Applications
The prototype has proven successful, so we are proceeding with the construction of a functional version of our system. We will encapsulate all logic into a separate package (module) and create a standardized "assembly line" (Application Factory) for its reliable construction.

- Routes, configuration, models, DB object, etc., are divided into `multiple python files (modules)`, and the application becomes a `package`.
  - `module` = any .py file from which we can import functions, classes, variables ..
  - `package` (an organized folder) = folders with .py files and an `__init__.py` file.
    - `subpackage` (a nested package) = a package inside another package.
- We have `folders` for HTML, static files, and elements.
- We access the application via the **Application Factory**.

- Suitable for:
  - Simple & single-purpose applications.
  - Applications with a unified `common environment`.
  - `Testing and prototyping`.


**Application structure**:
- my_project/
    - **wsgi.py**
    - **app/** - organized folder / package
        - **_ _ init _ _.py** - the `__init__.py` file "marks" the "/app" folder as a package
        - routes.py
        - database.py
        - **models/** - subpackage with table models
            - **_ _ init _ _.py** - the "/models" folder is marked as a package (or rather, a subpackage)
            - **drone_operator.py** - file with the "Drone_Operator" table model
        - data/
            - test_data.py
        - templates/
        - static/
        - config/ 

- The file with our **SQLite DB** is in the default **instance** folder outside our application.

### 7.2.1. `__init__.py` Files for Packages & Imports:
- The `__init__.py` file tells Python: "`the folder I am in is a Python package`" = organized code with .py files (modules).

1.  If we `import` this package or any of its modules, the code in **`__init__.py`** is `automatically executed` after the import.

2.  ! `all imports` of our own files (modules) within the application `must be rewritten from **absolute** to **relative** paths` !
    - With a relative path, we are saying: "stay within this package and look for **these files / folders**".

### * **Absolute** and **Relative** Paths:
```python
from database import db # absolute path
from .database import db # RELATIVE path
```

### * **Imports** & **`__init__.py`**:
- When using an `__init__.py` file (in the "models" folder):
```python
from .models import Drone_Operator
from .models import Probe
from .models import Mission
```

- EXPLICITLY without using an `__init__.py` file (in the "models" folder):
```python
from .models.drone_operator import Drone_Operator
from .models.probe import Probe
from .models.mission import Mission
```

### * Example of using "_ _ init _ _.py" for the "MODELS" module

In [ ]:
from .drone_operator import Drone_Operator

# if we have more table models, we can import them here as well - e.g.:

# from .probe import Probe
# from .mission import Mission

In [ ]:
# app/models/drone_operator.py

from ..database import db # import of 'db' object from a "higher" directory - .. = one level up

"""
DB table with four columns (id, pilot_name, email, callsign)
"""

class Drone_Operator(db.Model): # DB table class
    __tablename__ = "Drone_Operators" # DB table name

    id = db.Column(db.Integer, primary_key=True) 
    pilot_name = db.Column(db.String(20), nullable=False) 
    email = db.Column(db.String(20), nullable=False)
    callsign = db.Column(db.String(20), nullable=False)

    def __repr__(self):
        return f"Drone_Operator {self.pilot_name})"

### * Nesting routes in a function

In [ ]:
# app/routes.py

from flask import Flask, jsonify, make_response
from .data.test_data import probe_data


def register_routes(app): 
    @app.route("/earth") 
    def mission_earth():
        return """
            <h1>Mission</h1>
            <p>Welcome to Earth !</p>
        """
    
    @app.route("/mars")
    def mission_mars():
        return """
            <h1>Mission</h1>
            <p>Welcome to Mars !</p>
        """
    
    @app.route("/venus")
    def mission_venus():
        return """
            <h1>Mission</h1>
            <p>Welcome to Venus !</p>
        """
    
    @app.route("/probe_velocities", methods=["GET"])
    def probe_velocities():
        # Data passed to jsonify is obtained from a database, file, sensors ...
        probe_velocities = {"Velocities": [probe["velocity"] for probe in probe_data]}
        return jsonify(probe_velocities), 200


    @app.route("/probe_info/<int:probe_id>", methods=["GET"]) # we can use a parameter in the URL
    def probe_detail(probe_id):
        # Data passed to jsonify is obtained from a database, file, sensors ...
        if not 1 <= probe_id <= len(probe_data): # check if "probe_id" is within a valid range
            return jsonify({"error": "Probe ID is out of range. Not found."}), 404
        
        probe = probe_data[probe_id - 1]
        return jsonify(ID_Probe=probe_id, Details=probe), 200
        

    @app.route('/probe_mothership', methods=['GET'])
    def probe_mothership():
        # setting the BODY (response + status code using jsonify)
        response = make_response(jsonify(
            {"message": "No data to fetch, mothership empty, systems still running"}
            ), 202)
    
        # setting HEADERS
        response.headers["Cache-Control"] = "no-store" # cache setting
        response.headers["Content-Language"] = "en" # language setting

        # setting COOKIES
        response.set_cookie(
            key="session_mothership",
            value="50",
            max_age=3600,              # 1 hour
            httponly=True,
            secure=True,
            samesite="Lax"
        )
        return response
    
    # .. and other routes - e.g., using data from the database

---

### 7.2.2. The Ecosystem - Application Factory & Trigger
- The application factory ecosystem consists of two parts:

  - **`App Factory`** - a regular function that creates (and configures) the application and returns its instance.
    - = we place it in the `__init__.py` file.
  - **`Trigger`** - a launcher that imports the "application" instance and starts it.
    - = we place it in the `wsgi.py` file.
- Separation of application assembly and configuration (app factory) from its actual execution (trigger).

### 7.2.3. The Application Factory in `__init__.py`
- **`__init__.py`** is the **`entry point`** to our application:

  1.  Here we create our `own application` within a function.
  2.  We perform the `configuration`.
  3.  We initialize `plugins` (e.g., database).
  4.  We add `routes / Blueprints`.
  5.  We create everything for which we need the `application context` (e.g., the structure of tables in the db).
  
- As a result, function returns a `**battle-ready** application`.

In [ ]:
# app/__init__.py

# the function creates our application

from .config.app_config_class import DBConfig # import configuration - RELATIVE PATH !
from flask import Flask
from .database import db  # import 'db' object - RELATIVE PATH !
from .models import Drone_Operator  # import table models - RELATIVE PATH !
from .routes import register_routes # import routes - RELATIVE PATH !


def app_factory():

    app = Flask(__name__)

    # 1. CONFIGURATION
    app.config.from_object(DBConfig) 

    # 2. PLUGIN INITIALIZATION - if we have extensions for the "app" object - we add them after configuration
    db.init_app(app) # connecting "Flask app" with "SQLAlchemy db"

    # 3. REGISTRATION (Blueprints & Routes) - we add routes after plugins
    register_routes(app) # connecting routes

    # ------------
    # if we need to perform actions "within the application context", we do them last
    
    # 4. APPLICATION CONTEXT - actions that need to know the configuration and location 
    with app.app_context(): # use "app_context()
        db.create_all()  # creates all defined tables (models) - IF NOT EXISTS
        if not Drone_Operator.query.first(): # check if there are records in the DB, if not, insert the first pilot
            db.session.add(Drone_Operator(pilot_name="George Freedom", email="george.freedom@vast_space.com", callsign="Athamantis"))
            db.session.commit()

    # any other actions into the DB

    return app # returns "app" as an assembled and configured object

### 7.2.4. Trigger - in WSGI.py
A file named `wsgi` facilitates the deployment of the application to a production server.

1.  `Development server` (default Flask server):
    - We use it during development and start it with: `app.run()`.

2.  `Production server` (e.g., Gunicorn, uWSGI, or Waitress):
    - We use it for deployment and hosting - it does NOT run the `python wsgi.py` file.
    - We tell it where to find our application instance (the assembled and configured app object):
      - `filename:variable_name`
      - Example of starting with Gunicorn: `gunicorn --workers 4 --bind 0.0.0.0:8000 wsgi:app`
      - = this tells Gunicorn: "Find the file `wsgi.py`, inside it find the variable `app`, and serve it on port 8000."

* Many hosting platforms (Platform-as-a-Service) like Render, Heroku, Google App Engine, etc., are designed to detect a Python application and `automatically look for a file named wsgi.py`.
  - = when we name our startup file `wsgi.py` (and the application object **app** is available in it), we facilitate deployment (the platform knows how to start the application on its own) and often no further configuration is needed.

In [ ]:
# wsgi.py

# starts the application
"""
**advantage** of the __init__ file: 
- simple import of the application factory object 

= we write: "from app import app_factory" = from the application folder, import the function that creates, 
                                            configures, and initializes the application
= NOT: "from app.__init__ import app_factory" !

"""


from app import app_factory # import of the "app_factory" object - ABSOLUTE path !


app = app_factory()

if __name__ == "__main__":
    app.run(host='0.0.0.0')

### 7.2.5. `app.run()` Settings
1.  `app.run() without parameters`:
    - The Flask server starts at the address `127.0.0.1` (or **localhost**).
    - This is an address that means "this computer".
    - = the application is accessible only from the same computer it is running on.
2.  `app.run() with host='0.0.0.0' setting`:
    - **`0.0.0.0`** is a special address that tells the server: "Listen on all available network interfaces of this computer."
    - This includes not only `127.0.0.1`, `but mainly the IP address of our Wi-Fi or Ethernet card`.
- The application becomes **visible to other devices** on our local network!

#### **What is it good for?**
- `testing on mobile devices`: how the application looks on a phone or tablet - just open a browser on the mobile and enter our computer's IP address and port.
- `presenting a project` to colleagues within a team: we tell them the IP address where they can see the "running" application.
- `working with Docker or virtual machines`: when running Flask inside a container, the **`0.0.0.0`** setting is necessary to allow port forwarding from your computer to the container.

### **Practise**

**Assembly Line Construction**
We have a simple, functional prototype application for monitoring an atmospheric processor, but it lacks modularity for future expansion. Our task is to refactor this prototype into a professional, scalable structure using an `application factory`.


**Preparation:**

We have this unorganized `processor_monitor.py` file with a recommended default configuration:
```python
# processor_monitor.py

from flask import Flask


app = Flask(__name__)
# --- Configuration ---
app.config['SECRET_KEY'] = 'a-secret-key-for-the-processor'
app.config['DEBUG'] = True


@app.route("/")
def index():
    return "<h1>Atmospheric Processor Monitor</h1><p>Status: Online</p>"

@app.route("/data")
def get_data():
    return {"pressure": "102.5 kPa", "oxygen_level": "20.9%"}

if __name__ == "__main__":
    app.run()
```

**Assignment:**
We will create a new "Assembly Line" project structure and move the code from `processor_monitor.py` into it:

1.  **"Assembly Line" structure:**
- processor_project/
  - wsgi.py
  - app/
    - _ _ init _ _.py
    - config.py
    - routes.py

2.  **`config.py`:** Move the configuration into the configuration file.
3.  **`routes.py`:** Move the routes and wrap them in a function.
4.  **Build the Application Factory** = in `app/__init__.py`: create a `create_app()` function that creates an instance of `app`, loads the configuration from `config.py`, and registers the routes from `routes.py`.
5.  **Create `wsgi.py`:** Write a startup script that imports and starts the application factory.

### **Project (Homework): Upgrading the Core Control System**

**Mission:** Our current control system is functional, but it's a monolithic prototype. As the mission expands and tasks become more complex, it becomes cluttered and difficult to maintain. Our task is to perform the first phase of refactoring and rebuild the core of our application into a professional, scalable architecture using an application factory.


**Basic application structure:**
- my_project/
    - **wsgi.py**
    - **app/**
        - **_ _ init _ _.py**
        - routes.py
        - forms.py
        - database.py
        - **models/** 
            - **_ _ init _ _.py**
            - **crewmember.py**
        - data/
            - **_ _ init _ _.py**
            - asteroid.py
            - crew.py
        - templates/
        - static/
        - config/
    - **/instance**
        - **my_database.db**


Because our application has become a package with modules (and subpackages), the file with our **SQLite DB** will be created `**again** in the **instance** folder inside our project folder` - the application folder **/app** has become the **root** folder and the DB will be created "next to" it = in the project folder.


**1. Upgrading the Core Control System (Application Factory):** * Create a `wsgi.py` file, which will serve as the external launcher for our mission.
* Move all the logic for creating and configuring the application from the `my_app.py` file to a `create_app()` function in the `app/__init__.py` file = this is how we create the **application factory**.
* Modify `wsgi.py` to import and run the application from this factory.

**2. Standardizing Data Schemas:**
* Create a new **package** `app/models/`.
* Move the `CrewMember` model definition to a separate file `app/models/crew_member.py`.
* Create `app/models/__init__.py` (turning the folder into a package) and import the table model.
* Turn our `app/data` folder into a **package**.
* Create `__init__.py` for the `data` folder and import our older data about the asteroid and the crew.
* Modify the data imports in the files and in the application factory accordingly = we will use `relative path specification`.

**3. Systems Check:**
* After completing the modifications, test that the application still starts and all existing routes and functions work as they should.

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom